In [3]:
suppressPackageStartupMessages({
    source('./comparison/third_parties/symphony/vignettes/libs.R') # imports
    source('./comparison/third_parties/symphony/vignettes/utils.R') # color definitions and plotting functions
})

In [4]:
source("./preprocess/utils.R")

Attaching SeuratObject

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:dplyr':

    combine, intersect, setdiff, union


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Atta

In [5]:
library(Seurat)
library(SeuratDisk)
library(symphony)

In [6]:
cat_data <- function(rna, adt){
    counts <- rbind(rna[['rna']]@counts, adt[['adt']]@counts)
    adata <- CreateSeuratObject(counts)
    return(adata)
}


In [9]:
config <- parseTOML("./configs/data.toml")[['wnn_cellmask']]
subset_names <- basename(config$raw_data_dirs[1:8])
print(subset_names)
subset_ids <- sapply(seq_along(subset_names) - 1, toString)
input_dirs <- pj("./result/wnn_cellmask/inputs/default/predict/", paste0("subset_", subset_ids))

[1] "p1_0" "p2_0" "p3_0" "p4_0" "p5_0" "p6_0" "p7_0" "p8_0"


In [8]:
pp_dir <- "./data/processed/wnn_cellmask/"

In [41]:
z_list <- list()
cell_name_list <- list()
label_list <- list()
is_label <- T
subset_name_list <- list()
S <- length(subset_names)
for (i in seq_along(subset_names)) {
    subset_name <- subset_names[i]
    z_dir    <- pj(input_dirs[i], "x", "rna")
    fnames <- dir(path = z_dir, pattern = ".csv$")
    fnames <- str_sort(fnames, decreasing = F)
    z_subset_list <- list()
    N <- length(fnames)
    for (n in seq_along(fnames)) {
        message(paste0("Loading Subset ", i, "/", S, ", File ", n, "/", N))
        z_subset_list[[n]] <- read.csv(file.path(z_dir, fnames[n]), header = F)
    }
    z_list[[subset_name]] <- bind_rows(z_subset_list)

    cell_name_list[[subset_name]] <- read.csv(pj(pp_dir, paste0("subset_", subset_ids[i]),
        "cell_names.csv"), header = T)[, 2]
    subset_name_list[[subset_name]] <- rep(subset_name, length(cell_name_list[[subset_name]]))
}

rna <- t(data.matrix(bind_rows(z_list))) # pseudo rna counts
length(rna)
print(length(do.call("c", unname(cell_name_list))))
colnames(rna) <- do.call("c", unname(cell_name_list))
rownames(rna) <- paste0("rna-", seq_len(nrow(rna)))
obj <- CreateSeuratObject(counts = rna, assay = "rna")

Loading Subset 1/8, File 1/24



Loading Subset 1/8, File 2/24

Loading Subset 1/8, File 3/24

Loading Subset 1/8, File 4/24

Loading Subset 1/8, File 5/24

Loading Subset 1/8, File 6/24

Loading Subset 1/8, File 7/24

Loading Subset 1/8, File 8/24

Loading Subset 1/8, File 9/24

Loading Subset 1/8, File 10/24

Loading Subset 1/8, File 11/24

Loading Subset 1/8, File 12/24

Loading Subset 1/8, File 13/24

Loading Subset 1/8, File 14/24

Loading Subset 1/8, File 15/24

Loading Subset 1/8, File 16/24

Loading Subset 1/8, File 17/24

Loading Subset 1/8, File 18/24

Loading Subset 1/8, File 19/24

Loading Subset 1/8, File 20/24

Loading Subset 1/8, File 21/24

Loading Subset 1/8, File 22/24

Loading Subset 1/8, File 23/24

Loading Subset 1/8, File 24/24

Loading Subset 2/8, File 1/16

Loading Subset 2/8, File 2/16

Loading Subset 2/8, File 3/16

Loading Subset 2/8, File 4/16

Loading Subset 2/8, File 5/16

Loading Subset 2/8, File 6/16

Loading Subset 2/8, File 7/16

Loading Subset 2/8, File 8/16

Loading Subset 2/8, File

[1] 187162650

[1] 46213


In [42]:
z_list <- list()
cell_name_list <- list()
label_list <- list()
is_label <- T
subset_name_list <- list()
S <- length(subset_names)
for (i in seq_along(subset_names)) {
    subset_name <- subset_names[i]
    z_dir    <- pj(input_dirs[i], "x", "adt")
    fnames <- dir(path = z_dir, pattern = ".csv$")
    fnames <- str_sort(fnames, decreasing = F)
    z_subset_list <- list()
    N <- length(fnames)
    for (n in seq_along(fnames)) {
        message(paste0("Loading Subset ", i, "/", S, ", File ", n, "/", N))
        z_subset_list[[n]] <- read.csv(file.path(z_dir, fnames[n]), header = F)
    }
    z_list[[subset_name]] <- bind_rows(z_subset_list)

    cell_name_list[[subset_name]] <- read.csv(pj(pp_dir, paste0("subset_", subset_ids[i]),
        "cell_names.csv"), header = T)[, 2]
    subset_name_list[[subset_name]] <- rep(subset_name, length(cell_name_list[[subset_name]]))
}

rna <- t(data.matrix(bind_rows(z_list)))  # pseudo rna counts
length(rna)
print(length(do.call("c", unname(cell_name_list))))
colnames(rna) <- do.call("c", unname(cell_name_list))
rownames(rna) <- paste0("rna-", seq_len(nrow(rna)))
obj_adt <- CreateSeuratObject(counts = rna, assay = "adt")

Loading Subset 1/8, File 1/24

Loading Subset 1/8, File 2/24

Loading Subset 1/8, File 3/24



Loading Subset 1/8, File 4/24

Loading Subset 1/8, File 5/24

Loading Subset 1/8, File 6/24

Loading Subset 1/8, File 7/24

Loading Subset 1/8, File 8/24

Loading Subset 1/8, File 9/24

Loading Subset 1/8, File 10/24

Loading Subset 1/8, File 11/24

Loading Subset 1/8, File 12/24

Loading Subset 1/8, File 13/24

Loading Subset 1/8, File 14/24

Loading Subset 1/8, File 15/24

Loading Subset 1/8, File 16/24

Loading Subset 1/8, File 17/24

Loading Subset 1/8, File 18/24

Loading Subset 1/8, File 19/24

Loading Subset 1/8, File 20/24

Loading Subset 1/8, File 21/24

Loading Subset 1/8, File 22/24

Loading Subset 1/8, File 23/24

Loading Subset 1/8, File 24/24

Loading Subset 2/8, File 1/16

Loading Subset 2/8, File 2/16

Loading Subset 2/8, File 3/16

Loading Subset 2/8, File 4/16

Loading Subset 2/8, File 5/16

Loading Subset 2/8, File 6/16

Loading Subset 2/8, File 7/16

Loading Subset 2/8, File 8/16

Loading Subset 2/8, File 9/16

Loading Subset 2/8, File 10/16

Loading Subset 2/8, Fil

[1] 10351712

[1] 46213


In [43]:
adata <- cat_data(obj, obj_adt)
adata

Warning message:
"Non-unique features (rownames) present in the input matrix, making unique"


An object of class Seurat 
4274 features across 46213 samples within 1 assay 
Active assay: RNA (4274 features, 0 variable features)

In [44]:
ref_adata <- adata[, adata[['orig.ident']]=='B0']
ref_adata

An object of class Seurat 
4274 features across 6142 samples within 1 assay 
Active assay: RNA (4274 features, 0 variable features)

In [47]:
reference = symphony::buildReference(
    ref_adata@assays$RNA@counts,                   # reference expression (genes by cells)
    ref_adata@meta.data,              # reference metadata (cells x attributes)
    vars = c('orig.ident'),         # variable(s) to integrate over
    K = 100,                   # number of Harmony soft clusters
    verbose = TRUE,            # display verbose output
    do_umap = TRUE,            # run UMAP and save UMAP model to file
    do_normalize = TRUE,      # perform log(CP10k) normalization on reference expression
    vargenes_method = 'vst',   # variable gene selection method: 'vst' or 'mvp'
    vargenes_groups = 'orig.ident', # metadata column specifying groups for variable gene selection within each group
    topn = 3617,               # number of variable genes (per group)
    theta = 2,                 # Harmony parameter(s) for diversity term
    d = 32,                    # number of dimensions for PCA
    save_uwot_path = './result/wnn_cellmask/symphony_obo/default/train/', # file path to save uwot UMAP model
    additional_genes = NULL    # vector of any additional genes to force include
)

Normalizing

Finding variable genes using vst method



Total 3617 genes for downstream steps

Scaling and PCA

Running Harmony integration

Harmony 1/20

Harmony 2/20

Harmony converged after 2 iterations

Computing reference compression terms

Running UMAP

File already exists at that path... overwriting...

Saved uwot model



In [52]:
write.table(t(reference$Z_corr), './result/wnn_cellmask/symphony_obo/default/predict/subset_0/z/joint/00.csv', sep=',', row.names =F, col.names = F)

In [ ]:
# umap_labels = cbind(ref_adata@meta.data, reference$umap$embedding)
# fig.size(3, 5)
# plotBasic(umap_labels, title = 'Reference', color.by = 'l1')

In [53]:
for(i in c(1:7)){
    query_adata <- adata[, adata[['orig.ident']]==paste0('B', i)]
    query = mapQuery(query_adata@assays$RNA@counts,             # query gene expression (genes x cells)
                 query_adata@meta.data,        # query metadata (cells x attributes)
                 reference,             # Symphony reference object
                 vars = NULL,           # Query batch variables to harmonize over (NULL treats query as one batch)
                 do_normalize = TRUE,  # perform log(CP10k) normalization on query (set to FALSE if already normalized)
                 do_umap = FALSE)
    dir.create(paste0('./MIRACLE/result/wnn_cellmask/symphony_obo/default/predict/subset_',i,'/z/joint/'), recursive=T)
    write.table(t(query$Z), paste0('./MIRACLE/result/wnn_cellmask/symphony_obo/default/predict/subset_',i,'/z/joint/00.csv'), sep=',', row.names =F, col.names = F)
}

Normalizing

Scaling and synchronizing query gene expression

Found 3617 out of 3617 reference variable genes in query dataset

Project query cells using reference gene loadings

Clustering query cells to reference centroids

Correcting query batch effects

All done!

Normalizing

Scaling and synchronizing query gene expression

Found 3617 out of 3617 reference variable genes in query dataset

Project query cells using reference gene loadings

Clustering query cells to reference centroids

Correcting query batch effects

All done!

Normalizing

Scaling and synchronizing query gene expression

Found 3617 out of 3617 reference variable genes in query dataset

Project query cells using reference gene loadings

Clustering query cells to reference centroids

Correcting query batch effects

All done!

Normalizing

Scaling and synchronizing query gene expression

Found 3617 out of 3617 reference variable genes in query dataset

Project query cells using reference gene loadings

Clustering que

# offline

In [ ]:
ref_adata <- adata

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding miscellaneous information for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results

Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding miscellaneous information for ADT

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



An object of class Seurat 
3841 features across 46326 samples within 1 assay 
Active assay: RNA (3841 features, 0 variable features)

In [ ]:
dir.create('./results/wnn_cellmask/symphony_offline/default/train', recursive = T)

In [ ]:
reference = symphony::buildReference(
    adata@assays$RNA@counts,                   # reference expression (genes by cells)
    adata@meta.data,              # reference metadata (cells x attributes)
    vars = c('orig.ident'),         # variable(s) to integrate over
    K = 100,                   # number of Harmony soft clusters
    verbose = TRUE,            # display verbose output
    do_umap = TRUE,            # run UMAP and save UMAP model to file
    do_normalize = TRUE,      # perform log(CP10k) normalization on reference expression
    vargenes_method = 'vst',   # variable gene selection method: 'vst' or 'mvp'
    vargenes_groups = 'orig.ident', # metadata column specifying groups for variable gene selection within each group
    topn = 3841,               # number of variable genes (per group)
    theta = 2,                 # Harmony parameter(s) for diversity term
    d = 32,                    # number of dimensions for PCA
    save_uwot_path = './results/wnn_cellmask/symphony_offline/default/train/uwot_model_1', # file path to save uwot UMAP model
    additional_genes = NULL    # vector of any additional genes to force include
)

Normalizing

Finding variable genes using vst method

Total 3841 genes for downstream steps

Scaling and PCA

Running Harmony integration

Harmony 1/20

Harmony 2/20

Harmony 3/20

Harmony 4/20

Harmony 5/20

Harmony converged after 5 iterations

Computing reference compression terms

Running UMAP

Saved uwot model



In [ ]:
for(i in c(0:7)){
    a <- reference[, reference[['orig.ident']]==paste0('B', i)]
    dir.create(paste0('./results/wnn_cellmask/symphony_offline/default/predict/subset_',i), recursive = T)
    write.table(t(a$Z_corr), paste0('./results/wnn_cellmask/symphony_offline/default/predict/subset_',i,'00.csv'), sep=',', row.names =F, col.names = F)
}